In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 

In [3]:
!pip install transformers 

In [4]:
!python -m pip install huggingface_hub

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np



tokenizer = AutoTokenizer.from_pretrained("yirifiai1/BERT_Regulatory_Text_Classification_01")
model = AutoModelForSequenceClassification.from_pretrained("yirifiai1/BERT_Regulatory_Text_Classification_01")


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:

# # Define the id2label dictionary
# id2label = {idx: label for idx, label in enumerate(model.config.id2label)}

# def get_answer(text, model, tokenizer, id2label, threshold=0.3):
#     encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
#     encoding = {k: v.to(model.device) for k, v in encoding.items()}
    
#     with torch.no_grad():
#         outputs = model(**encoding)
#     logits = outputs.logits
    
#     sigmoid = torch.nn.Sigmoid()
#     probs = sigmoid(logits.squeeze().cpu())
    
#     predictions = (probs >= threshold).numpy().astype(int)
    
#     predicted_labels = {id2label[idx]: prob.item() for idx, prob in enumerate(probs) if predictions[idx] == 1}
    
#     return predicted_labels



In [22]:
# Define the id2label dictionary from the model configuration
id2label = model.config.id2label

def get_answer(text, model, tokenizer, id2label, threshold=0.4):
    # Tokenize and encode the input text
    encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=256)
    encoding = {k: v.to(model.device) for k, v in encoding.items()}
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**encoding)
    logits = outputs.logits
    
    # Apply sigmoid to get probabilities
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    
    # Apply threshold to get binary predictions
    predictions = (probs >= threshold).numpy().astype(int)
    
    # Map predictions to labels
    predicted_labels = [id2label[idx] for idx, prob in enumerate(probs) if predictions[idx] == 1]
    predicted_labels_prob = {id2label[idx]: prob.item() for idx, prob in enumerate(probs) if predictions[idx] == 1}
    
    return predicted_labels

In [23]:
# Example usage
text = " and going money and crypto bides text aml in vas"

predicted_labels = get_answer(text, model, tokenizer, id2label)
print(predicted_labels)


['aml', 'cft']


In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score

# Evaluation function
def evaluate(y_true, y_pred, labels):
    # Calculate subset accuracy (exact match ratio)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Subset Accuracy: {accuracy:.3f}')
    
    # Calculate F1 score
    
    f1_micro = f1_score(y_true, y_pred, average='micro')
    print(f'F1 Score (Micro): {f1_micro:.3f}')
    
    # Calculate AUC-ROC
    auc_roc = roc_auc_score(y_true, y_pred, average='micro')
    print(f'AUC-ROC: {auc_roc:.3f}')

In [25]:
df = pd.read_json("/kaggle/input/gpt-finetune-regulatory-text/regulatory_text_test.jsonl",lines=True)
#df = df.iloc[:5]

In [26]:
import json

# Initialize an empty set to store unique labels
labels_set = set()

# Read the JSONL file
with open('/kaggle/input/gpt-finetune-regulatory-text/regulatory_text_test.jsonl', 'r') as file:  # Replace with your file path
    for line in file:
        data = json.loads(line)
        # Extract the assistant's response
        assistant_content = data['messages'][2]['content'].split(", ")
        # Add each label to the set
        labels_set.update(assistant_content)

# Convert the set to a sorted list
labels = sorted(list(labels_set))

# Read the JSONL file
data = []
with open('/kaggle/input/gpt-finetune-regulatory-text/regulatory_text_test.jsonl', 'r') as file:  # Replace with your file path
    for line in file:
        data.append(json.loads(line))

# Extract content and create a list of dictionaries with binary topic columns
rows = []
for entry in data:
    user_content = entry['messages'][1]['content']
    assistant_content = entry['messages'][2]['content'].split(", ")

    row = {'Content': user_content}
    for topic in labels:
        row[topic] = 1 if topic in assistant_content else 0

    rows.append(row)

# Create DataFrame
df_onehot = pd.DataFrame(rows)
y_true = df_onehot[labels].values
#y_true = df_onehot.iloc[:5][labels].values

In [27]:
y_true

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [28]:
from transformers import pipeline
from tqdm import tqdm



def predict(test, model_name, labels):
    print(model_name)
    y_pred = []
    data_for_df = []  # List to store data for the DataFrame

    for i in tqdm(range(len(test))):
        system_prompt = test.iloc[i]["messages"][0]["content"]
        user_prompt = test.iloc[i]["messages"][1]["content"] # main text
        actual_label_list = test.iloc[i]["messages"][2]["content"]   # Ensure this is taken from `test`
        
        result = get_answer(user_prompt, model, tokenizer, id2label)  # result: list[string]
        print(result)
        y_pred.append(result)
        
        # Prepare the data for the DataFrame
        actual_labels_str = actual_label_list  # this is already a string
        result_str = ", ".join(result)  # Convert the list of labels to a comma-separated string
        data_for_df.append((user_prompt, actual_labels_str, result_str))

    # Convert predictions to binary matrix
    def parse_predictions(predictions, labels):
        y_pred = np.zeros((len(predictions), len(labels)), dtype=int)
        for i, pred_list in enumerate(predictions):
            for label in pred_list:
                if label in labels:
                    y_pred[i, labels.index(label)] = 1
        return y_pred

    y_pred_binary = parse_predictions(y_pred, labels)
    
    # Create the DataFrame
    df_compare = pd.DataFrame(data_for_df, columns=["user_prompt", "actual_labels", "result"])
    
    
    return y_pred_binary, df_compare

2024-09-04 16:29:15.327520: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 16:29:15.327663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 16:29:15.494483: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [29]:
X_test = df
model_name = "bert"
print(labels)

['aml', 'audit', 'authority', 'beneficiary information', 'cdd', 'cft', 'circular', 'client', 'compliance', 'conduct', 'controls', 'criminal', 'disclosure', 'dlt', 'due diligence', 'entities', 'guideline', 'identity', 'insider dealing', 'investor', 'investor protection', 'kyc', 'legal', 'legislation', 'license', 'monitoring', 'operator', 'oversight', 'pep', 'person', 'policies', 'policy', 'record keeping', 'regulations', 'regulator', 'reporting', 'requirements', 'risk assessment', 'risk management', 'risks', 'rulebook', 'rules', 'safeguard', 'sanctions', 'screening', 'suspicious transactions', 'transaction', 'transactions', 'transfer', 'travel rule', 'vasp', 'vasps', 'virtual assets']


In [30]:
y_pred, df_compare = predict(X_test, model_name, labels)

bert


  0%|          | 1/806 [00:00<09:30,  1.41it/s]

['virtual assets', 'requirements', 'license', 'regulator']


  0%|          | 2/806 [00:01<07:28,  1.79it/s]

['virtual assets', 'requirements', 'guideline', 'license', 'regulator']


...

['rulebook']


100%|█████████▉| 805/806 [06:04<00:00,  2.35it/s]

['rules']


100%|██████████| 806/806 [06:05<00:00,  2.21it/s]

['rules']


In [31]:
evaluate(y_true, y_pred, labels)

Subset Accuracy: 0.816
F1 Score (Micro): 0.954
AUC-ROC: 0.970


In [32]:
df_compare.head()

,user_prompt,actual_labels,result
0,the sfc is issuing this circular to highlight ...,"license, regulator, virtual assets, circular, ...","virtual assets, requirements, license, regulator"
1,the sfc has published the following guidelines...,"license, regulator, virtual assets, requiremen...","virtual assets, requirements, guideline, licen..."
2,(3) prevention of money laundering and terrori...,"license, cft, entities, virtual assets, aml, g...","virtual assets, aml, cft, guideline, license"
3,this circular supersedes the circular on virtu...,"circular, virtual assets","virtual assets, circular"
4,sfc authorised va funds should meet the applic...,requirements,requirements


In [34]:
df_compare.rename(columns = {"result":"bert_result"}, inplace=True)
df_compare.to_csv("df_compare_bert.csv")

In [2]:
import pandas as pd

In [15]:
df = pd.read_csv("/kaggle/input/df-compare-all-models/df_compare_all_models.csv")

In [16]:
df.head()

,Unnamed: 0,user_prompt,actual_labels,llama_4bit,Llama_16bit,bert_result,gpt4o-mini
0,0,the sfc is issuing this circular to highlight ...,"license, regulator, virtual assets, circular, ...","virtual assets, requirements, circular, licens...","vasps, conduct, compliance, requirements, regu...","virtual assets, requirements, license, regulator","license, circular, regulator, virtual assets, ..."
1,1,the sfc has published the following guidelines...,"license, regulator, virtual assets, requiremen...","virtual assets, requirements, guideline, licen...","vasps, conduct, investor, person, code of conduct","virtual assets, requirements, guideline, licen...","license, regulator, virtual assets, requiremen..."
2,2,(3) prevention of money laundering and terrori...,"license, cft, entities, virtual assets, aml, g...","virtual assets, aml, entities, cft, guideline","aml, cft, guideline, sfc, virtual asset servic...","virtual assets, aml, cft, guideline, license","license, cft, entities, virtual assets, aml, g..."
3,3,this circular supersedes the circular on virtu...,"circular, virtual assets",virtual assets,"vasps, regulations, circular","virtual assets, circular","circular, virtual assets"
4,4,sfc authorised va funds should meet the applic...,requirements,requirements,"vasps, conduct, investor, person, code of conduct",requirements,"{""requirements""}"


In [17]:
# Convert any non-string values to an empty string for the label columns
label_columns = ['llama_4bit', 'Llama_16bit', 'bert_result', 'gpt4o-mini']

# Fill NaN or None with empty strings
df[label_columns] = df[label_columns].fillna("")

# Split the label columns into sets for comparison
df['llama_4bit_set'] = df['llama_4bit'].apply(lambda x: set(x.split(", ")))
df['Llama_16bit_set'] = df['Llama_16bit'].apply(lambda x: set(x.split(", ")))
df['bert_result_set'] = df['bert_result'].apply(lambda x: set(x.split(", ")))
df['gpt4o-mini_set'] = df['gpt4o-mini'].apply(lambda x: set(x.split(", ")))
df['actual_labels_set'] = df['actual_labels'].apply(lambda x: set(x.split(", ")))

# Find common labels across all models
df['common_labels'] = df.apply(lambda row: row['actual_labels_set'].intersection(
    row['llama_4bit_set'], row['bert_result_set'], row['gpt4o-mini_set']), axis=1)

# Find labels each model missed from the original
df['missed_by_llama_4bit'] = df.apply(lambda row: row['actual_labels_set'] - row['llama_4bit_set'], axis=1)
df['missed_by_bert_result'] = df.apply(lambda row: row['actual_labels_set'] - row['bert_result_set'], axis=1)
df['missed_by_gpt4o-mini'] = df.apply(lambda row: row['actual_labels_set'] - row['gpt4o-mini_set'], axis=1)

# Find labels each model added that were not in the original
df['new_by_llama_4bit'] = df.apply(lambda row: row['llama_4bit_set'] - row['actual_labels_set'], axis=1)
df['new_by_bert_result'] = df.apply(lambda row: row['bert_result_set'] - row['actual_labels_set'], axis=1)
df['new_by_gpt4o-mini'] = df.apply(lambda row: row['gpt4o-mini_set'] - row['actual_labels_set'], axis=1)

# Keep only the required columns and convert sets to comma-separated strings
result_df = df[['user_prompt', 'actual_labels', 'llama_4bit', 'bert_result', 'gpt4o-mini', 'common_labels',
                'missed_by_llama_4bit',  'missed_by_bert_result', 'missed_by_gpt4o-mini',
                'new_by_llama_4bit', 'new_by_bert_result', 'new_by_gpt4o-mini']]

# Convert the sets to strings for better readability
result_df = result_df.applymap(lambda x: ', '.join(x) if isinstance(x, set) else x)

# Display the first few rows of the result
result_df.head()


/tmp/ipykernel_34/1244753670.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result_df = result_df.applymap(lambda x: ', '.join(x) if isinstance(x, set) else x)


,user_prompt,actual_labels,llama_4bit,bert_result,gpt4o-mini,common_labels,missed_by_llama_4bit,missed_by_bert_result,missed_by_gpt4o-mini,new_by_llama_4bit,new_by_bert_result,new_by_gpt4o-mini
0,the sfc is issuing this circular to highlight ...,"license, regulator, virtual assets, circular, ...","virtual assets, requirements, circular, licens...","virtual assets, requirements, license, regulator","license, circular, regulator, virtual assets, ...","requirements, virtual assets, regulator, license",,circular,,,,
1,the sfc has published the following guidelines...,"license, regulator, virtual assets, requiremen...","virtual assets, requirements, guideline, licen...","virtual assets, requirements, guideline, licen...","license, regulator, virtual assets, requiremen...","guideline, regulator, license, virtual assets,...",,,,,,
2,(3) prevention of money laundering and terrori...,"license, cft, entities, virtual assets, aml, g...","virtual assets, aml, entities, cft, guideline","virtual assets, aml, cft, guideline, license","license, cft, entities, virtual assets, aml, g...","aml, guideline, virtual assets, cft",license,entities,,,,
3,this circular supersedes the circular on virtu...,"circular, virtual assets",virtual assets,"virtual assets, circular","circular, virtual assets",virtual assets,circular,,,,,
4,sfc authorised va funds should meet the applic...,requirements,requirements,requirements,"{""requirements""}",,,,requirements,,,"{""requirements""}"


In [18]:
# Function to sort the labels alphabetically
def sort_labels(label_set):
    return ', '.join(sorted(label_set.split(', ')))

# Apply the sorting function to each relevant column
sorted_result_df = result_df.copy()
columns_to_sort = [
    'actual_labels', 'llama_4bit', 'bert_result', 'gpt4o-mini', 'common_labels',
                'missed_by_llama_4bit', 'missed_by_bert_result', 'missed_by_gpt4o-mini',
                'new_by_llama_4bit', 'new_by_bert_result', 'new_by_gpt4o-mini'
]

for column in columns_to_sort:
    sorted_result_df[column] = sorted_result_df[column].apply(sort_labels)

# Save the sorted dataframe to a new CSV file
output_file_path = 'df_compare_all_models_sorted_02.csv'
sorted_result_df.to_csv(output_file_path, index=False)